In [2]:
import warnings
warnings.filterwarnings(action='ignore')

import torch
import time 
from sklearn.model_selection import KFold,StratifiedKFold
import torch.nn as nn

from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter

from gensim.models.phrases import Phrases
from gensim.models import Word2Vec

from src.train_utils import set_seed, ModelSave, get_torch_device, EarlyStop, TrainParams
from src.metric import  multi_cls_metrics,multi_cls_log
from src.dataset.tokenizer import GensimTokenizer
from iflytek_app.dataset import MixDataset
from iflytek_app.models import TextcnnMixup
from iflytek_app.process import train_process,test_process,result_process, kfold_inference

device = get_torch_device()
set_seed()

No GPU available, using the CPU instead.


## Load Data & word2vec

In [3]:
c2v = GensimTokenizer( Word2Vec.load('./checkpoint/char_min1_win5_sg_d100'))
w2v = GensimTokenizer(Word2Vec.load('./checkpoint/phrase_min1_win5_sg_d100'))
w2v.init_vocab()
c2v.init_vocab()
phraser = Phrases.load('./checkpoint/phrase_tokenizer')
df, label2idx = train_process()

                id           l1           l2          len
count  4199.000000  4199.000000  4199.000000  4199.000000
mean   2099.000000     8.969278    37.087878    46.057156
std    1212.291219     4.576621    79.204914    79.332999
min       0.000000     2.000000     1.000000     4.000000
25%    1049.500000     5.000000     6.000000    15.000000
50%    2099.000000     8.000000    12.000000    22.000000
75%    3148.500000    12.000000    26.000000    36.000000
max    4198.000000    32.000000   946.000000   961.000000
{'14784131 14858934 14784131 14845064': 0, '14852788 14717848 15639958 15632020': 1, '14844856 14724258 14925237 14854807': 2, '14925756 15639967 14853254 14728639': 3, '14844593 14924945': 4, '15709098 14716590 14924703 14779559': 5, '14726332 14728344 14854542 14844591': 6, '14858934 15636660 15704193 14849963': 7, '15710359 14847407 14845602 14859696': 8, '14794687 14782344': 9, '15630486 15702410 14718849 15709093': 10, '15632285 15706536 14721977 14925219': 11, '147829

## Train 5-Fold Model


In [4]:
log_steps = 10
save_steps = 20
kfold=5
tp = TrainParams(
    epoch_size=30,
    lr=1e-3,
    loss_fn=nn.CrossEntropyLoss(),
    max_seq_len=1000,
    batch_size=64,
    dropout_rate=0.5,
    label_size = len(label2idx),
    vocab_size = w2v.vocab_size,
    embedding_dim = w2v.embedding_size + c2v.embedding_size,
    embedding1=c2v.embedding, 
    embedding2 =w2v.embedding,
    num_train_steps=int(df.shape[0]/kfold * (kfold-1)), 
    filter_size=70,
    kernel_size_list = [2,3,4,5],
    hidden_size = 100,
    mixup_alpha =0.5,
    early_stop_params = {
        'monitor':'f1_micro',
        'mode':'max',
        'min_delta': 0,
        'patience':5,
        'verbose':False
    },
    scheduler_params={'mode': 'max',
                     'factor': 0.3,
                     'patience': 2,
                     'verbose': True,
                     'threshold':0.0001,
                     'threshold_mode':'rel',
                     'cooldown':0,
                     'min_lr':1e-6}
)

In [14]:
minor_id = df.loc[df['label'].isin(['14784131 14858934 14784131 14845064',
                                    '14852788 14717848 15639958 15632020']),'id'].values.tolist()

In [27]:
kf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=24)
for fold,(train_index, valid_index) in enumerate(kf.split(df.values, df[['label']].values)):
    train, valid = df.iloc[train_index.tolist()+minor_id], df.iloc[valid_index]

    train_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx, 
                               train['name'].values, train['description'].values, train['label'].values)
    valid_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx, 
                               valid['name'].values, valid['description'].values, valid['label'].values)
    train_sampler = RandomSampler(train_dataset)
    valid_sampler = SequentialSampler(valid_dataset)
    train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=tp.batch_size)
    valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=tp.batch_size)
    
    CKPT = './checkpoint/textcnn_mixup_v3/k{}'.format(fold)
    saver = ModelSave(CKPT, continue_train=False)
    tb = SummaryWriter(CKPT)
    es = EarlyStop(**tp.early_stop_params)
    global_step = 0
    saver.init()
    model = TextcnnMixup(tp)
    optimizer, scheduler = model.get_optimizer()
    
    
    for epoch_i in range(tp['epoch_size']):
        if global_step==1:
            print(model)
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10}  | {'Elapsed':^9}")
        print("-"*60)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()
        total_loss, batch_loss, batch_counts = 0, 0, 0

        model.train()
        for step, batch in enumerate(train_loader):
            global_step +=1
            batch_counts +=1

            #Forward propogate
            model.zero_grad()
            feature = {k:v.to(device) for k, v in batch.items()}
            logits = model(feature)
            loss = model.compute_loss(feature, logits)
            batch_loss += loss.item()
            total_loss += loss.item()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()

            # Log steps for train loss logging
            if (step % log_steps == 0 and step != 0) or (step == len(train_loader) - 1):
                time_elapsed = time.time() - t0_batch
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^9} | {time_elapsed:^9.2f}")
                tb.add_scalar('loss/batch_train', batch_loss / batch_counts, global_step=global_step)
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

            # Save steps for ckpt saving and dev evaluation
            if (step % save_steps == 0 and step != 0) or (step == len(train_loader) - 1):
                val_metrics = multi_cls_metrics(model, valid_loader, device)
                for key, val in val_metrics.items():
                    tb.add_scalar(f'metric/{key}', val, global_step=global_step)
                avg_train_loss = total_loss / step
                tb.add_scalars('loss/train_valid',{'train_loss': avg_train_loss,
                                                    'valid_loss': val_metrics['val_loss']}, global_step=global_step)
                saver(total_loss / step, val_metrics['val_loss'], epoch_i, global_step, model, optimizer, scheduler)

        # On Epoch End: calcualte train & valid loss and log overall metrics
        time_elapsed = time.time() - t0_epoch
        val_metrics = multi_cls_metrics(model, valid_loader, device)
        avg_train_loss = total_loss / step
        scheduler.step(val_metrics['f1_micro'])
        print("-"*70)
        print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_metrics['val_loss']:^10.6f} | {time_elapsed:^9.2f}")
        multi_cls_log(epoch_i, val_metrics)
        print("\n")
        if es.check(val_metrics):
            break 

./checkpoint/textcnn_mixup_v3/k0 model cleaned
 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   10    |   3.766691   |     -     |   18.30  
   1    |   20    |   2.496191   |     -     |   16.99  
   1    |   30    |   2.664559   |     -     |   19.01  
   1    |   40    |   2.425688   |     -     |   13.18  
   1    |   50    |   2.182534   |     -     |   18.85  
   1    |   52    |   2.181654   |     -     |   2.23   
----------------------------------------------------------------------
   1    |    -    |   2.759358   |  2.154078  |   94.38  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   1    |  12.623%  |  71.351%  |  24.508%  |     7.073%      |   12.623%    |  35.000%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
-

   8    |   30    |   0.456751   |     -     |   18.54  
   8    |   40    |   0.417259   |     -     |   12.92  
   8    |   50    |   0.394228   |     -     |   18.89  
   8    |   52    |   0.292721   |     -     |   1.84   
----------------------------------------------------------------------
   8    |    -    |   0.768130   |  0.945344  |   89.08  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   8    |  61.265%  |  89.880%  |  69.490%  |     68.883%     |   61.265%    |  71.786%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
   8    |  71.786%  |     -     |     -     |     71.786%     |   71.786%    |  71.786%  




 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
--------------------------------------

   3    |   52    |   1.032626   |     -     |   2.02   
----------------------------------------------------------------------
   3    |    -    |   1.583805   |  1.177506  |   91.55  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   3    |  42.902%  |  84.953%  |  53.885%  |     53.060%     |   42.902%    |  64.286%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
   3    |  64.286%  |     -     |     -     |     64.286%     |   64.286%    |  64.286%  




 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   4    |   10    |   1.673096   |     -     |   17.15  
   4    |   20    |   1.661960   |     -     |   14.95  
   4    |   30    |   0.948999   |

  11    |   10    |   0.983507   |     -     |   16.65  
  11    |   20    |   0.971984   |     -     |   14.87  
  11    |   30    |   0.279922   |     -     |   19.06  
  11    |   40    |   0.234080   |     -     |   12.98  
  11    |   50    |   0.195045   |     -     |   18.84  
  11    |   52    |   0.176657   |     -     |   2.06   
----------------------------------------------------------------------
  11    |    -    |   0.538119   |  0.902921  |   90.58  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
  11    |  64.478%  |  90.918%  |  73.719%  |     69.075%     |   64.478%    |  76.310%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
  11    |  76.310%  |     -     |     -     |     76.310%     |   76.310%    

  18    |   30    |   0.077351   |     -     |   18.70  
  18    |   40    |   0.096614   |     -     |   12.78  
  18    |   50    |   0.084598   |     -     |   18.68  
  18    |   52    |   0.102283   |     -     |   1.98   
----------------------------------------------------------------------
  18    |    -    |   0.432500   |  0.787366  |   90.59  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
  18    |  65.583%  |  90.930%  |  75.286%  |     71.635%     |   65.583%    |  78.214%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
  18    |  78.214%  |     -     |     -     |     78.214%     |   78.214%    |  78.214%  




./checkpoint/textcnn_mixup_v3/k2 model cleaned
 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  

----------------------------------------------------------------------
   7    |    -    |   0.806046   |  0.987882  |   90.43  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   7    |  53.764%  |  88.653%  |  62.410%  |     63.550%     |   53.764%    |  73.333%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
   7    |  73.333%  |     -     |     -     |     73.333%     |   73.333%    |  73.333%  




 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   8    |   10    |   1.258912   |     -     |   16.44  
   8    |   20    |   1.215499   |     -     |   14.62  
   8    |   30    |   0.365607   |     -     |   19.15  
   8    |   40    |   0.374557   |

   5    |   10    |   1.561760   |     -     |   17.11  
   5    |   20    |   1.514815   |     -     |   15.01  
   5    |   30    |   0.903693   |     -     |   18.61  
   5    |   40    |   0.886417   |     -     |   12.88  
   5    |   50    |   0.820022   |     -     |   18.64  
   5    |   52    |   0.623625   |     -     |   2.14   
----------------------------------------------------------------------
   5    |    -    |   1.147617   |  1.103790  |   90.20  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   5    |  48.533%  |  88.696%  |  59.906%  |     55.935%     |   48.533%    |  69.286%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
   5    |  69.286%  |     -     |     -     |     69.286%     |   69.286%    

  12    |   52    |   0.230231   |     -     |   2.07   
----------------------------------------------------------------------
  12    |    -    |   0.666350   |  0.974090  |   89.37  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
  12    |  64.796%  |  90.054%  |  71.495%  |     75.658%     |   64.796%    |  75.833%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
  12    |  75.833%  |     -     |     -     |     75.833%     |   75.833%    |  75.833%  




 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
  13    |   10    |   1.208628   |     -     |   16.49  
  13    |   20    |   1.199479   |     -     |   15.05  
  13    |   30    |   0.228352   |

   3    |   10    |   2.312584   |     -     |   16.26  
   3    |   20    |   1.935342   |     -     |   15.11  
   3    |   30    |   1.336354   |     -     |   18.96  
   3    |   40    |   1.247945   |     -     |   13.16  
   3    |   50    |   1.184018   |     -     |   18.68  
   3    |   52    |   1.306396   |     -     |   2.28   
----------------------------------------------------------------------
   3    |    -    |   1.636304   |  1.325813  |   90.29  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   3    |  40.805%  |  79.921%  |  51.075%  |     50.607%     |   40.805%    |  60.429%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
   3    |  60.429%  |     -     |     -     |     60.429%     |   60.429%    

  10    |   52    |   0.338803   |     -     |   2.24   
----------------------------------------------------------------------
  10    |    -    |   0.714191   |  0.974971  |   90.60  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
  10    |  61.915%  |  83.832%  |  67.556%  |     67.274%     |   61.915%    |  75.209%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
  10    |  75.209%  |     -     |     -     |     75.209%     |   75.209%    |  75.209%  




 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
  11    |   10    |   1.337463   |     -     |   16.12  
  11    |   20    |   1.138903   |     -     |   15.02  
  11    |   30    |   0.325157   |

In [28]:
test = test_process()
test_dataset = test_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx,
                                         test['name'].values, test['description'].values)
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset, sampler=test_sampler, batch_size=tp.batch_size)

In [29]:
result = kfold_inference(test_loader, tp, TextcnnMixup, './checkpoint/textcnn_mixup_v3', 5, device)
result['pred'] = result['pred_avg']
result_process(result, label2idx, './submit/textcnn_mixup_v3_5fold_avg.csv')